In [89]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
data =  pd.read_csv("mobile_price_classification.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        1841 non-null   object 
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [6]:
data = data.dropna()

In [15]:
data['n_cores'] = data['n_cores'].replace('x', 1)
data['n_cores'] = data['n_cores'].astype('float')

### Models


In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split

In [74]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV, RandomizedSearchCV

In [75]:
from sklearn.metrics import f1_score, accuracy_score

### BAGGING

In [76]:
train, test = train_test_split(data, random_state=2, train_size=0.8)

In [77]:
train.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1277,1528,0,2.4,0,2,0,37,0.2,156,8.0,...,306,564,733,15,0,14,1,1,0,0
1926,1122,0,0.5,0,0,1,40,0.3,156,7.0,...,675,1163,1456,9,3,20,1,1,0,1
1729,1982,0,1.1,0,12,0,24,0.1,129,7.0,...,764,1873,3511,16,14,15,1,1,1,3
1463,1147,0,0.8,1,11,0,28,0.7,158,6.0,...,1085,1134,2627,18,3,15,1,0,1,2
961,1807,1,0.5,1,2,1,61,0.1,134,6.0,...,1734,1968,3726,8,5,15,1,0,0,3


In [78]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1].values, data.iloc[:, -1].values, random_state=2, train_size=0.8)

In [79]:
X_train.shape, y_train.shape

((1472, 20), (1472,))

In [80]:
np.random.seed(10)
a = np.random.dirichlet(np.ones(4),size=20)
List = [i for i in a]

k = [i for i in range(20)]
k.append('sqrt')
k.append('log2')

k1 = ['linear', 'poly', 'sigmoid']
d = [3,4]
g = ['scale','auto']


In [99]:
a = DecisionTreeClassifier(random_state=32)

In [109]:
a.set_params = {'max_features': k}
a.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 32,
 'splitter': 'best'}

In [124]:
bag = 5
models = [DecisionTreeClassifier(random_state=32), DecisionTreeClassifier(random_state=43), SVC(), KNeighborsClassifier(), GaussianNB()]
parameter = [{'max_features': k}, {'max_features': k}, {'kernel': k1, 'gamma' : g, 'degree': d},{'n_neighbors': [1,2,3,4,5,6,7,8,9,10,11,12, 15,16,20]}, {'priors': List}]

index = np.arange(X_train.shape[0])
train_s =  int(X_train.shape[0] * 0.8)

np.random.seed(42)
fit_models = []
for i, model in enumerate(models):
    np.random.shuffle(index)
    train_data = X_train[index[:train_s], :]
    train_label = y_train[index[:train_s]]
    
#     grid = RandomizedSearchCV(model, param_distributions= parameter[i] ,   cv=5, n_jobs=-1)
    grid = GridSearchCV(model, param_grid= parameter[i] ,   cv=5, n_jobs=-1)
    grid.fit(X_train, y_train)
    print(grid.best_score_)
    print(grid.best_params_)
    
    model.set_params(**grid.best_params_)
    #print(model.get_params())
    fit_models.append(model.fit(X_train, y_train))

0.8383373688458434
{'max_features': 19}
0.8383535108958838
{'max_features': 17}
0.9694407932664593
{'degree': 3, 'gamma': 'scale', 'kernel': 'linear'}
0.9327683615819209
{'n_neighbors': 15}
0.8037149775164304
{'priors': array([0.20576641, 0.20085726, 0.33937841, 0.25399793])}


In [119]:
fit_models[2].get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [147]:
def predict(models, X_test):
    Matrix = pd.DataFrame()
    for i, model in enumerate(models):
        predicted = model.predict(X_test)
        Matrix[i] = predicted
    
    return  Matrix.mode(axis=1).values.reshape(-1)

In [150]:
(predict(models, X_test) == y_test).mean()

0.907859078590786